In [2]:
from fastapi import FastAPI, HTTPException, Query
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import PlainTextResponse
from pydantic import BaseModel
from typing import Dict, Any, Union, List, Optional
import inspect
import uvicorn
import nest_asyncio
import socket
import pandas as pd
import json
from threading import Thread


# Import from Rules.ipynb
%run Rules.ipynb

nest_asyncio.apply()

app = FastAPI(title="Business Rules Engine")
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"]
)

# ---------- Data Functions ----------
DATA_FUNCTIONS = {
    "get_customers": get_customers,
    "get_accounts": get_accounts,
    "get_transactions": get_transactions,
    "get_cust_acc": get_cust_acc,
    "get_cust_tran": get_cust_tran,
    "get_acc_tran": get_acc_tran,
    "get_cust_acc_tran": get_cust_acc_tran
}

# ---------- Rule Management ----------
RULE_DESCRIPTIONS = {
    f"rule_{i:02d}": func.__doc__ or f"Rule {i} description"
    for i, func in enumerate(RULE_REGISTRY.values(), 1)
}

TEMP_RULES = {}

class RuleModel(BaseModel):
    rule_id: str
    description: str
    function_name: str
    column: str
    condition: str
    value: Any
    code: str
    extra: Dict[str, Any] = {}

@app.get("/rules", response_class=PlainTextResponse)
def list_rules():
    predefined_rules = {
        "rule_01": "Customers without any associated account",
        "rule_02": "Accounts with no transaction history",
        "rule_03": "Transactions above ₹1,00,000",
        "rule_04": "Accounts activated before 2020",
        "rule_05": "Transactions with zero amount",
        "rule_06": "Customers with more than one account",
        "rule_07": "Accounts inactive for more than 1 year",
        "rule_08": "Transactions on non-active accounts",
        "rule_09": "Customers missing city details",
        "rule_10": "Transactions that occurred before account activation",
        "rule_11": "5+ transactions within a 1-minute window",
        "rule_12": "10+ small (< ₹100) transactions per account",
        "rule_13": "Customers transacting across multiple accounts",
        "rule_14": "Invalid rows with mismatched customer IDs",
        "rule_15": "Customers with duplicate names"
    }
    lines = ["📋 Business Rules:\n"]
    for rule_id, desc in predefined_rules.items():
        lines.append(f"🔹 {rule_id}: {desc} (Predefined)")
    for rule_id, rule_data in TEMP_RULES.items():
        desc = rule_data.get("description", "Dynamic rule")
        lines.append(f"🆕 {rule_id}: {desc} (Dynamic)")
    return "\n".join(lines)
    

import numpy as np

@app.get("/run_rule/{rule_id}")
def run_rule_by_id(rule_id: str):
    if rule_id in RULE_REGISTRY:
        try:
            result = run_rule(rule_id)
            return {
                "rule_id": rule_id,
                "description": RULE_DESCRIPTIONS.get(rule_id, ""),
                "result": result
            }
        except Exception as e:
            raise HTTPException(status_code=500, detail=f"Error running rule: {e}")
    if rule_id in TEMP_RULES:
        rule = TEMP_RULES[rule_id]
        fn = DATA_FUNCTIONS[rule["function_name"]]
        df = fn()
        local_vars = {"df": df.copy()}
        try:
            exec(rule["code"], {}, local_vars)
            result_df = local_vars.get("df", df)
            result_json = json.loads(result_df.replace({np.nan: None}).to_json(orient="records"))
            return {
                "rule_id": rule_id,
                "description": rule["description"],
                "result": result_json
            }
        except Exception as e:
            raise HTTPException(status_code=500, detail=f"Dynamic rule error: {e}")
    raise HTTPException(status_code=404, detail="Rule not found")


@app.post("/add_rule")
def add_dynamic_rule(rule: RuleModel):
    if rule.rule_id in RULE_DESCRIPTIONS or rule.rule_id in TEMP_RULES:
        raise HTTPException(status_code=400, detail="Rule ID already exists")
    if rule.function_name not in DATA_FUNCTIONS:
        raise HTTPException(status_code=400, detail="Function not found")
    TEMP_RULES[rule.rule_id] = rule.dict()
    return {"message": "Rule added successfully"}

@app.get("/functions", response_class=PlainTextResponse)
def list_functions(level: int = Query(0, ge=0, le=2), function_name: Optional[str] = Query(None)):
    if level == 0:
        description = """
🟢 Level 0 - Basic Data Retrieval:
These are the foundational functions that fetch raw, unjoined data from the database. They are typically used as a base for analysis or filtering before applying any business logic or joining operations.
They offer complete flexibility with multiple filtering options.

Functions:
- get_customers
- get_accounts
- get_transactions
"""
        if function_name == "get_customers":
            description += """
get_customers:
- customer_id: ID(s) of customers to retrieve.
- name: Exact or partial match for customer name.
- city: Filter by city name(s).
- update_date: Filter specific dates (YYYY-MM-DD).
- exact_match: False allows partial matching.
- min_update_date / max_update_date: Filter by a range of update dates.
"""
        elif function_name == "get_accounts":
            description += """
get_accounts:
- account_no: One or more account numbers.
- account_type: Account type (e.g., savings, current).
- customer_id: Filter by customer IDs.
- account_status: e.g., active, closed.
- activation_date: Specific dates.
- exact_match: Set to False for partial search.
- min_activation_date / max_activation_date: Filter by range.
"""
        elif function_name == "get_transactions":
            description += """
get_transactions:
- transaction_id: Filter by transaction ID(s).
- account_no: Account numbers involved.
- customer_id: Customer IDs involved.
- amount: Exact value(s).
- min_amount / max_amount: Range filters.
- transaction_time: Timestamps (exact or list).
- min_transaction_time / max_transaction_time: Time window filter.
"""
        return description

    elif level == 1:
        description = """
🟡 Level 1 - Joined Data Retrieval:
These functions combine two related entities such as customers and accounts, or accounts and transactions. They're commonly used for identifying relationships or cross-checking data between two tables.
Ideal for use cases where a simple join operation is needed to understand dependencies or relationships in the data.

Functions:
- get_cust_acc
- get_cust_tran
- get_acc_tran
"""
        if function_name == "get_cust_acc":
            description += """
get_cust_acc:
Joins customer and account data on specified keys.

Parameters:
- join_type (str): Type of join (e.g., 'left', 'inner'). Default is 'left'.
- customer_on (str or List[str]): Column(s) in customer data to join on.
- account_on (str or List[str]): Column(s) in account data to join on.
- customer_filters (dict): Filters to apply if cust DataFrame is not provided.
- account_filters (dict): Filters to apply if acc DataFrame is not provided.
- cust (pd.DataFrame): Optional. If provided, used directly as customers data.
- acc (pd.DataFrame): Optional. If provided, used directly as accounts data.
"""
        elif function_name == "get_cust_tran":
            description += """
get_cust_tran:
Joins customer and transaction data on specified keys.

Parameters:
- join_type (str): Type of SQL join ('left', 'inner', etc.). Default is 'left'.
- customer_on (str or List[str]): Key(s) from the customer table to join on.
- transaction_on (str or List[str]): Key(s) from the transaction table to join on.
- customer_filters (dict): Filters for get_customers if no cust DataFrame is passed.
- transaction_filters (dict): Filters for get_transactions if no tran DataFrame is passed.
- cust (pd.DataFrame): Optional. Pre-filtered customer DataFrame.
- tran (pd.DataFrame): Optional. Pre-filtered transaction DataFrame.
"""
        elif function_name == "get_acc_tran":
            description += """
get_acc_tran:
Joins account and transaction data on specified keys.

Parameters:
- join_type (str): Type of join ('left', 'inner', etc.). Default is 'left'.
- account_on (str or List[str]): Key(s) from the account table.
- transaction_on (str or List[str]): Key(s) from the transaction table.
- account_filters (dict): Filters to apply on get_accounts() if acc not provided.
- transaction_filters (dict): Filters to apply on get_transactions() if tran not provided.
- acc (pd.DataFrame): Optional. Pre-filtered account DataFrame.
- tran (pd.DataFrame): Optional. Pre-filtered transaction DataFrame.
"""
        return description

    elif level == 2:
        return """
🔴 Level 2 -Super Function:
This advanced utility joins all three core tables: customers, accounts, and transactions. It allows detailed correlation and traceability across the entire customer lifecycle. This is highly useful for detecting anomalies, customer behavior analytics, and policy compliance.

Function:
- get_cust_acc_tran

get_cust_acc_tran:
Parameters:
- join_type (str): Type of join to apply ('left', 'inner', etc.)
- customer_filters (dict): Filters for customers if 'cust' not provided.
- account_filters (dict): Filters for accounts if 'acc' not provided.
- transaction_filters (dict): Filters for transactions if 'tran' not provided.
- customer_account_key (str or List[str]): Join key(s) between customers and accounts.
- account_transaction_key (str or List[str]): Join key(s) between accounts and transactions.
- cust (pd.DataFrame): Optional. Pre-filtered customers DataFrame.
- acc (pd.DataFrame): Optional. Pre-filtered accounts DataFrame.
- tran (pd.DataFrame): Optional. Pre-filtered transactions DataFrame.
"""

    return "Invalid level. Use level 0, 1, or 2."

class FunctionCall(BaseModel):
    function_name: str
    parameters: Dict[str, Any] = {}

@app.post("/run_function")
def run_function_with_parameters(body: FunctionCall):
    fn = DATA_FUNCTIONS.get(body.function_name)
    if not fn:
        raise HTTPException(status_code=404, detail="Function not found")

    params = body.parameters
    
    # Special handling for filters parameter
    if 'filters' in params and isinstance(params['filters'], str):
        try:
            params['filters'] = json.loads(params['filters'])
        except json.JSONDecodeError:
            raise HTTPException(
                status_code=400,
                detail="Invalid filters format. Should be valid JSON"
            )

    try:
        result = fn(**params)
        if hasattr(result, 'to_dict'):
            result_json = json.loads(result.replace({np.nan: None}).to_json(orient="records"))
            return result_json
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))



@app.delete("/delete_rule/{rule_id}")
def delete_rule(rule_id: str):
    if rule_id not in TEMP_RULES:
        raise HTTPException(status_code=404, detail="Rule ID not found")
    del TEMP_RULES[rule_id]
    return {"message": f"Rule '{rule_id}' deleted successfully"}

from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from fastapi.requests import Request

app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

@app.get("/")
def dashboard(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})


import nest_asyncio
import uvicorn
import socket
from threading import Thread

nest_asyncio.apply()

def find_open_port(start=8000):
    for port in range(start, 8100):
        with socket.socket() as s:
            if s.connect_ex(("127.0.0.1", port)) != 0:
                return port
    raise RuntimeError("No open ports")

def start_ui_server():
    port = find_open_port()
    print(f"🔗 Visit your dashboard at: http://127.0.0.1:{port}")
    uvicorn.run(app, host="127.0.0.1", port=port)

Thread(target=start_ui_server, daemon=True).start()


# ---------- Launch FastAPI Server ----------
def find_open_port(start=8000):
    for port in range(start, 8100):
        with socket.socket() as s:
            if s.connect_ex(("127.0.0.1", port)) != 0:
                return port
    raise RuntimeError("No open ports")

def start_server():
    port = find_open_port()
    print(f"\n✅ FastAPI running: http://127.0.0.1:{port}/docs")
    uvicorn.run(app, host="127.0.0.1", port=port)

Thread(target=start_server, daemon=True).start()

🔗 Visit your dashboard at: http://127.0.0.1:8001

✅ FastAPI running: http://127.0.0.1:8001/docs


INFO:     Started server process [15700]
INFO:     Started server process [15700]
INFO:     Waiting for application startup.
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8001): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


INFO:     127.0.0.1:55787 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:55787 - "GET /rules HTTP/1.1" 200 OK
INFO:     127.0.0.1:55787 - "GET /favicon.ico HTTP/1.1" 404 Not Found


Task exception was never retrieved
future: <Task finished name='Task-2' coro=<Server.serve() done, defined at C:\Users\prash\anaconda3\envs\Pandas_playground\Lib\site-packages\uvicorn\server.py:68> exception=SystemExit(1)>
Traceback (most recent call last):
  File "C:\Users\prash\anaconda3\envs\Pandas_playground\Lib\site-packages\uvicorn\server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prash\anaconda3\envs\Pandas_playground\Lib\asyncio\base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8001): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\prash\anaconda3\envs\Pandas_playground\Lib\threading.py", line 1075, in _bo

INFO:     127.0.0.1:55871 - "POST /run_function HTTP/1.1" 200 OK
INFO:     127.0.0.1:55882 - "POST /add_rule HTTP/1.1" 200 OK


C:\Users\prash\AppData\Local\Temp\ipykernel_15700\4258982569.py:125: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  TEMP_RULES[rule.rule_id] = rule.dict()


INFO:     127.0.0.1:55882 - "GET /rules HTTP/1.1" 200 OK
INFO:     127.0.0.1:55884 - "GET /run_rule/rule_16 HTTP/1.1" 200 OK


In [1]:
import os

# Create folders
os.makedirs("templates", exist_ok=True)
os.makedirs("static", exist_ok=True)

# Write professional Deloitte-themed HTML template
with open("templates/index.html", "w", encoding="utf-8") as f:
    f.write(r"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Deloitte | Business Rule Engine</title>
  <script src="https://cdn.tailwindcss.com"></script>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
  <style>
    :root {
      --deloitte-green: #86BC25;
      --deloitte-dark-green: #2E7D32;
      --deloitte-dark: #2C2C2C;
      --deloitte-light: #F5F5F5;
    }
    body {
      font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    .param-description {
      display: none;
      background-color: #f8fafc;
      border-left: 4px solid var(--deloitte-green);
      padding: 0.75rem;
      margin-top: 0.5rem;
      font-size: 0.875rem;
      border-radius: 0 4px 4px 0;
    }
    .function-card:hover {
      background-color: rgba(134, 188, 37, 0.05);
      border-left: 3px solid var(--deloitte-green);
    }
    .level-info {
      background-color: white;
      border-radius: 6px;
      padding: 1.25rem;
      margin-bottom: 1.5rem;
      box-shadow: 0 2px 8px rgba(0,0,0,0.05);
      border-top: 3px solid;
    }
    .level-0 {
      border-top-color: var(--deloitte-green);
    }
    .level-1 {
      border-top-color: #FFC72C;
    }
    .level-2 {
      border-top-color: #D50000;
    }
    .level-badge {
      display: inline-flex;
      align-items: center;
      padding: 0.25rem 0.75rem;
      border-radius: 12px;
      font-weight: 600;
      font-size: 0.75rem;
      margin-right: 0.75rem;
    }
    .badge-0 {
      background-color: var(--deloitte-green);
      color: white;
    }
    .badge-1 {
      background-color: #FFC72C;
      color: var(--deloitte-dark);
    }
    .badge-2 {
      background-color: #D50000;
      color: white;
    }
    .deloitte-header {
      background-color: white;
      border-bottom: 1px solid #e5e7eb;
      box-shadow: 0 2px 4px rgba(0,0,0,0.05);
    }
    .deloitte-btn-primary {
      background-color: var(--deloitte-green);
      color: white;
      transition: all 0.2s;
    }
    .deloitte-btn-primary:hover {
      background-color: var(--deloitte-dark-green);
      transform: translateY(-1px);
    }
    .deloitte-card {
      background: white;
      border-radius: 8px;
      box-shadow: 0 2px 6px rgba(0,0,0,0.05);
      border: 1px solid #e5e7eb;
    }
    .section-title {
      color: var(--deloitte-dark);
      position: relative;
      padding-left: 1rem;
    }
    .section-title:before {
      content: '';
      position: absolute;
      left: 0;
      top: 0;
      bottom: 0;
      width: 4px;
      background: var(--deloitte-green);
      border-radius: 2px;
    }
  </style>
</head>
<body class="bg-gray-50 text-gray-800">
<!-- Header -->
<header class="deloitte-header py-4 px-6 sticky top-0 z-10">
  <div class="max-w-7xl mx-auto flex justify-between items-center">
    <div class="flex items-center space-x-2">
      <svg width="32" height="32" viewBox="0 0 32 32" fill="none" xmlns="http://www.w3.org/2000/svg">
        <path d="M16 32C24.8366 32 32 24.8366 32 16C32 7.16344 24.8366 0 16 0C7.16344 0 0 7.16344 0 16C0 24.8366 7.16344 32 16 32Z" fill="#86BC25"/>
        <path d="M22.4 9.6001H9.6V22.4001H22.4V9.6001Z" fill="white"/>
      </svg>
      <h1 class="text-xl font-bold text-gray-800">Deloitte Business Rule Engine</h1>
    </div>
    <div class="text-sm text-gray-600">
      <i class="fas fa-user-circle mr-1"></i> Admin Dashboard
    </div>
  </div>
</header>

<main class="py-6 px-4 max-w-7xl mx-auto">
 <!-- Dashboard Overview -->
<div class="mb-8 deloitte-card p-6">
  <h2 class="text-2xl font-semibold mb-2 text-gray-800">Rule Engine Dashboard</h2>
  <p class="text-gray-600 mb-4">Manage and execute business rules across customer, account, and transaction data.</p>
  <div class="grid grid-cols-1 md:grid-cols-2 gap-4">
    <div class="bg-blue-50 p-4 rounded-lg border border-blue-100">
      <div class="flex items-center">
        <div class="p-2 rounded-full bg-blue-100 text-blue-600 mr-3">
          <i class="fas fa-database"></i>
        </div>
        <div>
          <h3 class="font-medium text-gray-700">Functions</h3>
          <p class="text-2xl font-bold">7</p>
        </div>
      </div>
    </div>
    <div class="bg-purple-50 p-4 rounded-lg border border-purple-100">
      <div class="flex items-center">
        <div class="p-2 rounded-full bg-purple-100 text-purple-600 mr-3">
          <i class="fas fa-layer-group"></i>
        </div>
        <div>
          <h3 class="font-medium text-gray-700">Levels</h3>
          <p class="text-2xl font-bold">3</p>
        </div>
      </div>
    </div>
  </div>
</div>

  <div class="grid grid-cols-1 lg:grid-cols-2 gap-6">
    <!-- Left Column -->
    <div class="space-y-6">
      <!-- Rules Section -->
      <div class="deloitte-card p-6">
        <h2 class="section-title text-xl font-semibold mb-4">📋 Business Rules</h2>
        <div class="mb-4">
          <pre id="rules" class="bg-gray-50 p-4 rounded overflow-auto text-sm whitespace-pre-wrap font-mono border border-gray-200 max-h-60"></pre>
        </div>
        <button onclick="fetchRules()" class="deloitte-btn-primary px-4 py-2 rounded-md font-medium">
          <i class="fas fa-sync-alt mr-2"></i>Refresh Rules
        </button>
      </div>

      <!-- Run Rule Section -->
      <div class="deloitte-card p-6">
        <h2 class="section-title text-xl font-semibold mb-4">▶️ Execute Rule</h2>
        <div class="flex items-center space-x-2 mb-4">
          <input id="rule_id_input" class="border px-3 py-2 rounded-md flex-grow" placeholder="Enter rule ID (e.g., rule_01)">
          <button onclick="runRule()" class="deloitte-btn-primary px-4 py-2 rounded-md font-medium">
            <i class="fas fa-play mr-1"></i>Run
          </button>
        </div>
 <div id="rule_output" class="bg-white p-4 rounded-md shadow overflow-auto">
  <table class="min-w-full table-auto text-sm w-full">
    <thead>
      <tr>
        <th class="w-1/4 text-left px-2 py-1 border-b">Account Number</th>
        <th class="w-1/4 text-left px-2 py-1 border-b">Account Type</th>
        <th class="w-1/4 text-left px-2 py-1 border-b">Customer ID</th>
        <th class="w-1/4 text-left px-2 py-1 border-b">Status</th>
      </tr>
    </thead>
    <tbody>
      <!-- Your data rows dynamically inserted here -->
    </tbody>
  </table>
</div>

      </div>
    </div>

    <!-- Right Column -->
    <div class="space-y-6">
      <!-- Add Rule Section -->
      <div class="deloitte-card p-6">
        <h2 class="section-title text-xl font-semibold mb-4">🆕 Create New Rule</h2>
        
        <div class="space-y-3 mb-4">
          <div class="level-info level-0">
            <div class="flex items-center mb-1">
              <span class="level-badge badge-0">Level 0</span>
              <h3 class="font-bold text-gray-800">Basic Data Retrieval</h3>
            </div>
            <p class="text-sm pl-8 text-gray-600">Foundational functions that fetch raw, unjoined data from the database. They are typically used as a base for analysis or filtering before applying any business logic or joining operations. They offer complete flexibility with multiple filtering options.</p>
          </div>
          
          <div class="level-info level-1">
            <div class="flex items-center mb-1">
              <span class="level-badge badge-1">Level 1</span>
              <h3 class="font-bold text-gray-800">Joined Data Retrieval</h3>
            </div>
            <p class="text-sm pl-8 text-gray-600">Functions that combine two related entities such as customers and accounts, or accounts and transactions. They're commonly used for identifying relationships or cross-checking data between two tables. Ideal for use cases where a simple join operation is needed to understand dependencies or relationships in the data.</p>
          </div>
          
          <div class="level-info level-2">
            <div class="flex items-center mb-1">
              <span class="level-badge badge-2">Level 2</span>
              <h3 class="font-bold text-gray-800">Super Function</h3>
            </div>
            <p class="text-sm pl-8 text-gray-600">Advanced utility that joins all three core tables: customers, accounts, and transactions. Allows detailed correlation and traceability across the entire customer lifecycle. Highly useful for detecting anomalies, customer behavior analytics, and policy compliance.</p>
          </div>
        </div>
        
        <form id="add_rule_form" class="grid grid-cols-1 gap-3">
          <div class="grid grid-cols-2 gap-3">
            <div>
              <label class="block text-sm font-medium text-gray-700 mb-1">Rule ID</label>
              <input name="rule_id" class="w-full border px-3 py-2 rounded-md" placeholder="rule_20">
            </div>
            <div>
              <label class="block text-sm font-medium text-gray-700 mb-1">Level</label>
              <select name="level" class="w-full border px-3 py-2 rounded-md">
                <option value="0">Level 0 (Basic Functions)</option>
                <option value="1">Level 1 (Joined Functions)</option>
                <option value="2">Level 2 (Super Functions)</option>
              </select>
            </div>
          </div>
          
          <div>
            <label class="block text-sm font-medium text-gray-700 mb-1">Description</label>
            <input name="description" class="w-full border px-3 py-2 rounded-md" placeholder="Rule description">
          </div>
          
          <div class="grid grid-cols-3 gap-3">
              <div>
                  <label class="block text-sm font-medium text-gray-700 mb-1">Function</label>
                  <select name="function_name" id="function_dropdown" onchange="populateColumns()" class="w-full border px-3 py-2 rounded-md">
                    <option value="">Select function</option>
                    <option value="get_customers">get_customers</option>
                    <option value="get_accounts">get_accounts</option>
                    <option value="get_transactions">get_transactions</option>
                    <option value="get_cust_acc">get_cust_acc</option>
                    <option value="get_cust_tran">get_cust_tran</option>
                    <option value="get_acc_tran">get_acc_tran</option>
                    <option value="get_cust_acc_tran">get_cust_acc_tran</option>
                  </select>
                </div>
             </div>
            <div>
            <div>
              <label class="block text-sm font-medium text-gray-700 mb-1">Column</label>
              <select name="column" id="column_dropdown" class="w-full border px-3 py-2 rounded-md">
                <option value="">Select column</option>
                </select>

            </div>
            <div>
              <label class="block text-sm font-medium text-gray-700 mb-1">Condition</label>
              <input name="condition" class="w-full border px-3 py-2 rounded-md" placeholder=">">
            </div>
          </div>
          
          <div>
            <label class="block text-sm font-medium text-gray-700 mb-1">Value</label>
            <input name="value" class="w-full border px-3 py-2 rounded-md" placeholder="1000">
          </div>
          
          <div>
            <label class="block text-sm font-medium text-gray-700 mb-1">Custom Code</label>
            <textarea name="code" class="w-full border px-3 py-2 rounded-md h-24" placeholder="Custom rule code using 'df' as DataFrame"></textarea>
          </div>
        </form>
        
        <button onclick="addRule()" class="deloitte-btn-primary w-full mt-3 px-4 py-2 rounded-md font-medium">
          <i class="fas fa-plus mr-2"></i>Add Rule
        </button>
      </div>
    </div>
  </div>

<!-- Function Section -->
<div class="bg-white p-6 rounded-lg border border-gray-200 shadow-md mt-4">
  <h2 class="text-xl font-semibold mb-4 text-gray-800">🧮 Data Functions</h2>

  <div class="grid grid-cols-1 md:grid-cols-3 gap-4">
    <!-- Column 1: L0 Functions -->
    <div class="bg-gray-50 p-4 rounded-lg border border-gray-200">
      <h3 class="text-sm font-semibold text-gray-700 mb-2">LEVEL 0 [Base Level Functions]</h3>
        
        <!-- get_customers -->
        <div class="function-card p-3 rounded-md cursor-pointer border border-gray-200" onclick="toggleDescription('get_customers_desc')">
          <div class="flex items-center gap-2">
            <span class="level-badge badge-0">LEVEL 0</span>
            <div class="font-semibold text-gray-800">get_customers</div>
          </div>
          <div class="text-sm text-gray-600 mt-1">Retrieves customer data with various filtering options</div>
          <div id="get_customers_desc" class="param-description mt-2 text-sm space-y-1 hidden">
            <div><strong>customer_id</strong>: ID(s) of customers to retrieve</div>
            <div><strong>name</strong>: Exact or partial match for customer name</div>
            <div><strong>city</strong>: Filter by city name(s)</div>
            <div><strong>update_date</strong>: Filter specific dates (YYYY-MM-DD)</div>
            <div><strong>exact_match</strong>: False allows partial matching</div>
            <div><strong>min_update_date/max_update_date</strong>: Filter by date range</div>
          </div>
        </div>

        <!-- get_accounts -->
        <div class="function-card p-3 rounded-md cursor-pointer border border-gray-200" onclick="toggleDescription('get_accounts_desc')">
          <div class="flex items-center gap-2">
            <span class="level-badge badge-0">LEVEL 0</span>
            <div class="font-semibold text-gray-800">get_accounts</div>
          </div>
          <div class="text-sm text-gray-600 mt-1">Retrieves account data with filtering capabilities</div>
          <div id="get_accounts_desc" class="param-description mt-2 text-sm space-y-1 hidden">
            <div><strong>account_no</strong>: One or more account numbers</div>
            <div><strong>account_type</strong>: Account type (e.g., savings, current)</div>
            <div><strong>customer_id</strong>: Filter by customer IDs</div>
            <div><strong>account_status</strong>: e.g., active, closed</div>
            <div><strong>activation_date</strong>: Specific dates</div>
            <div><strong>exact_match</strong>: Set to False for partial search</div>
            <div><strong>min_activation_date/max_activation_date</strong>: Filter by date range</div>
          </div>
        </div>

        <!-- get_transactions -->
        <div class="function-card p-3 rounded-md cursor-pointer border border-gray-200" onclick="toggleDescription('get_transactions_desc')">
          <div class="flex items-center gap-2">
            <span class="level-badge badge-0">LEVEL 0</span>
            <div class="font-semibold text-gray-800">get_transactions</div>
          </div>
          <div class="text-sm text-gray-600 mt-1">Retrieves transaction data with various filters</div>
          <div id="get_transactions_desc" class="param-description mt-2 text-sm space-y-1 hidden">
            <div><strong>transaction_id</strong>: Filter by transaction ID(s)</div>
            <div><strong>account_no</strong>: Account numbers involved</div>
            <div><strong>customer_id</strong>: Customer IDs involved</div>
            <div><strong>amount</strong>: Exact value(s)</div>
            <div><strong>min_amount/max_amount</strong>: Range filters</div>
            <div><strong>transaction_time</strong>: Timestamps (exact or list)</div>
            <div><strong>min_transaction_time/max_transaction_time</strong>: Time window filter</div>
          </div>
        </div>
      </div>

    <!-- Column 2 -->
      <div class="bg-gray-50 p-4 rounded-lg border border-gray-200">
      <h3 class="text-sm font-semibold text-gray-700 mb-2">LEVEL 1 [Joined Level Functions]</h3>

        <!-- get_cust_acc -->
        <div class="function-card p-3 rounded-md cursor-pointer border border-gray-200" onclick="toggleDescription('get_cust_acc_desc')">
          <div class="flex items-center gap-2">
            <span class="level-badge badge-1">LEVEL 1</span>
            <div class="font-semibold text-gray-800">get_cust_acc</div>
          </div>
          <div class="text-sm text-gray-600 mt-1">Joins customer and account data</div>
          <div id="get_cust_acc_desc" class="param-description mt-2 text-sm space-y-1 hidden">
            <div><strong>join_type</strong>: Type of join ('left', 'inner')</div>
            <div><strong>customer_on</strong>: Column(s) in customer data to join on</div>
            <div><strong>account_on</strong>: Column(s) in account data to join on</div>
            <div><strong>customer_filters</strong>: Filters for customer data</div>
            <div><strong>account_filters</strong>: Filters for account data</div>
            <div><strong>cust</strong>: Optional pre-filtered customer DataFrame</div>
            <div><strong>acc</strong>: Optional pre-filtered account DataFrame</div>
          </div>
        </div>

        <!-- get_cust_tran -->
        <div class="function-card p-3 rounded-md cursor-pointer border border-gray-200" onclick="toggleDescription('get_cust_tran_desc')">
          <div class="flex items-center gap-2">
            <span class="level-badge badge-1">LEVEL 1</span>
            <div class="font-semibold text-gray-800">get_cust_tran</div>
          </div>
          <div class="text-sm text-gray-600 mt-1">Joins customer and transaction data</div>
          <div id="get_cust_tran_desc" class="param-description mt-2 text-sm space-y-1 hidden">
            <div><strong>join_type</strong>: Type of join ('left', 'inner')</div>
            <div><strong>customer_on</strong>: Key(s) from customer table</div>
            <div><strong>transaction_on</strong>: Key(s) from transaction table</div>
            <div><strong>customer_filters</strong>: Filters for customers</div>
            <div><strong>transaction_filters</strong>: Filters for transactions</div>
            <div><strong>cust</strong>: Optional pre-filtered customer DataFrame</div>
            <div><strong>tran</strong>: Optional pre-filtered transaction DataFrame</div>
          </div>
        </div>

        <!-- get_acc_tran -->
        <div class="function-card p-3 rounded-md cursor-pointer border border-gray-200" onclick="toggleDescription('get_acc_tran_desc')">
          <div class="flex items-center gap-2">
            <span class="level-badge badge-1">LEVEL 1</span>
            <div class="font-semibold text-gray-800">get_acc_tran</div>
          </div>
          <div class="text-sm text-gray-600 mt-1">Joins account and transaction data</div>
          <div id="get_acc_tran_desc" class="param-description mt-2 text-sm space-y-1 hidden">
            <div><strong>join_type</strong>: Type of join ('left', 'inner')</div>
            <div><strong>account_on</strong>: Key(s) from account table</div>
            <div><strong>transaction_on</strong>: Key(s) from transaction table</div>
            <div><strong>account_filters</strong>: Filters for accounts</div>
            <div><strong>transaction_filters</strong>: Filters for transactions</div>
            <div><strong>acc</strong>: Optional pre-filtered account DataFrame</div>
            <div><strong>tran</strong>: Optional pre-filtered transaction DataFrame</div>
          </div>
        </div>
      </div>
      <!-- Column 3: L2 Functions (Moved here) -->
    <div class="bg-gray-50 p-4 rounded-lg border border-gray-200">
      <h3 class="text-sm font-semibold text-gray-700 mb-2">LEVEL 2 [Super Function]</h3>
      <div class="space-y-3">
        <div class="function-card p-3 rounded-md cursor-pointer border border-gray-200" onclick="toggleDescription('get_cust_acc_tran_desc')">
          <div class="flex items-center gap-2">
            <span class="level-badge badge-2">LEVEL 2</span>
            <div class="font-semibold text-gray-800">get_cust_acc_tran</div>
          </div>
          <div class="text-sm text-gray-600 mt-1">Joins customer, account, and transaction data</div>
          <div id="get_cust_acc_tran_desc" class="param-description mt-2 text-sm space-y-1 hidden">
            <div><strong>join_type</strong>: Type of join ('left', 'inner')</div>
            <div><strong>customer_filters</strong>: Filters for customers</div>
            <div><strong>account_filters</strong>: Filters for accounts</div>
            <div><strong>transaction_filters</strong>: Filters for transactions</div>
            <div><strong>customer_account_key</strong>: Join key(s) between customers and accounts</div>
            <div><strong>account_transaction_key</strong>: Join key(s) between accounts and transactions</div>
            <div><strong>cust</strong>: Optional pre-filtered customers DataFrame</div>
            <div><strong>acc</strong>: Optional pre-filtered accounts DataFrame</div>
            <div><strong>tran</strong>: Optional pre-filtered transactions DataFrame</div>
          </div>
        </div>
      </div>
    </div>
  </div>
</div>
<script>
function toggleDescription(id) {
  const el = document.getElementById(id);
  if (el) el.classList.toggle('hidden');
}
</script>

      
    <!-- ⚙️ Function Execution Card -->
<div class="bg-white p-6 rounded-lg border border-gray-200 shadow-md mt-6">
  <h2 class="text-xl font-semibold mb-4 text-gray-800">⚙️ Function Execution</h2>
  
  <!-- Function Dropdown -->
  <div class="mb-4">
    <label class="block text-sm font-medium text-gray-700 mb-2">Select Function</label>
    <select id="function_select" onchange="updateFunctionFields()" class="border border-gray-300 px-3 py-2 rounded-md w-full focus:outline-none focus:ring-2 focus:ring-blue-500">
      <option value="get_customers">get_customers</option>
      <option value="get_accounts">get_accounts</option>
      <option value="get_transactions">get_transactions</option>
      <option value="get_cust_acc">get_cust_acc</option>
      <option value="get_cust_tran">get_cust_tran</option>
      <option value="get_acc_tran">get_acc_tran</option>
      <option value="get_cust_acc_tran">get_cust_acc_tran</option>
    </select>
  </div>

  <!-- Dynamic Input Fields -->
  <div id="function_inputs" class="grid grid-cols-1 md:grid-cols-2 gap-4 mb-4">
    <!-- Populated dynamically via JS -->
  </div>

  <!-- Execute Button -->
  <button onclick="runFunction()" class="bg-green-400 text-white hover:bg-green-600 transition-colors w-full px-4 py-2 rounded-md font-medium">
    <i class="fas fa-play mr-2"></i> Execute Function
  </button>

  <!-- Output Section -->
  <div class="mt-6">
    <label class="block text-sm font-medium text-gray-700 mb-2">Results</label>
    <div id="function_output" class="bg-gray-50 p-4 rounded-md overflow-auto text-sm border border-gray-200 max-h-60">
      <!-- Output shown here -->
    </div>
  </div>
</div>

    </div>
  </div>

  <!-- Delete Rule Section -->
  <div class="mt-6 deloitte-card p-6">
    <h2 class="section-title text-xl font-semibold mb-4">❌ Rule Management</h2>
    <div class="flex items-center space-x-3">
      <input id="delete_rule_id" class="border px-3 py-2 rounded-md flex-grow" placeholder="Enter Rule ID to Delete">
      <button onclick="deleteRule()" class="deloitte-btn-primary px-4 py-2 rounded-md font-medium bg-red-600 hover:bg-red-700">
        <i class="fas fa-trash-alt mr-1"></i>Delete Rule
      </button>
    </div>
  </div>
</main>

<script>
  const functionParams = {
    get_customers: ["customer_id","name", "city", "update_date", "exact_match", "min_update_date", "max_update_date"],
    get_accounts: ["account_no", "account_type", "customer_id", "account_status", "activation_date", "exact_match", "min_activation_date", "max_activation_date"],
    get_transactions: ["transaction_id", "account_no", "customer_id", "amount", "transaction_time", "min_transaction_time", "max_transaction_time", "min_amount", "max_amount"],
    get_cust_acc: ["join_type", "customer_on", "account_on", "customer_filters", "account_filters"],
    get_cust_tran: ["join_type", "customer_on", "transaction_on", "customer_filters", "transaction_filters"],
    get_acc_tran: ["join_type", "account_on", "transaction_on", "account_filters", "transaction_filters"],
    get_cust_acc_tran: ["join_type", "customer_account_key", "account_transaction_key", "customer_filters", "account_filters", "transaction_filters"]
  };

  function toggleDescription(id) {
    const desc = document.getElementById(id);
    desc.style.display = desc.style.display === 'none' ? 'block' : 'none';
  }

  function fetchRules() {
    fetch('/rules')
      .then(res => res.text())
      .then(data => {
        document.getElementById("rules").textContent = data;
        // Update rules count
        const rules = data.split('\n').filter(line => line.trim().startsWith('rule_'));
        document.getElementById("rules-count").textContent = rules.length;
      });
  }

  function runRule() {
    const ruleId = document.getElementById("rule_id_input").value;
    if (!ruleId) return alert("Please enter a rule ID");
    
    fetch('/run_rule/' + ruleId)
      .then(res => res.json())
      .then(data => {
        displayTable("rule_output", data.result);
      })
      .catch(err => alert("Rule Error: " + err));
  }

    function runFunction() {
  const func = document.getElementById("function_select").value;
  const inputs = document.querySelectorAll("#function_inputs input, #function_inputs select");
  const params = {};

  inputs.forEach(input => {
    let value = input.value.trim();
    if (!value) return;

    // If comma-separated → turn into array
    if (value.includes(',')) {
      const items = value.split(',').map(v => v.trim());
      params[input.name] = items.map(v => {
        if (v.toLowerCase() === "true") return true;
        if (v.toLowerCase() === "false") return false;
        return isNaN(v) ? v : parseFloat(v);
      });
    } else {
      if (value.toLowerCase() === "true") {
        params[input.name] = true;
      } else if (value.toLowerCase() === "false") {
        params[input.name] = false;
      } else {
        params[input.name] = isNaN(value) ? value : parseFloat(value);
      }
    }
  });

  fetch('/run_function', {
    method: "POST",
    headers: { "Content-Type": "application/json" },
    body: JSON.stringify({ function_name: func, parameters: params })
  })
  .then(res => res.json())
  .then(data => displayTable("function_output", data))
  .catch(err => alert("Function Error: " + err));
}



  function displayTable(divId, data) {
    const div = document.getElementById(divId);
    if (!data || data.length === 0) {
      div.innerHTML = "<div class='text-center py-4 text-gray-500'>No data returned</div>";
      return;
    }

    const keys = Object.keys(data[0]);
    let html = "<div class='overflow-x-auto'><table class='min-w-full divide-y divide-gray-200'><thead class='bg-gray-100'><tr>";
    keys.forEach(key => {
      html += `<th scope='col' class='px-4 py-2 text-left text-xs font-medium text-gray-700 uppercase tracking-wider'>${key}</th>`;
    });
    html += "</tr></thead><tbody class='bg-white divide-y divide-gray-200'>";
    
    data.slice(0, 100).forEach((row, i) => {
      html += "<tr class='" + (i % 2 === 0 ? 'bg-white' : 'bg-gray-50') + "'>";
      keys.forEach(key => {
        const val = row[key];
        html += `<td class='px-4 py-2 text-sm ${typeof val === 'number' ? 'text-right font-mono' : 'text-left'}'>${val || ''}</td>`;
      });
      html += "</tr>";
    });
    html += "</tbody></table></div>";
    
    if (data.length > 100) {
      html += `<div class='text-xs text-gray-500 mt-2'>Showing 100 of ${data.length} rows</div>`;
    }
    
    div.innerHTML = html;
  }

  function updateFunctionFields() {
    const func = document.getElementById("function_select").value;
    const container = document.getElementById("function_inputs");
    container.innerHTML = "";
    (functionParams[func] || []).forEach(p => {
      const div = document.createElement("div");
      div.className = "space-y-1";
      
      const label = document.createElement("label");
      label.textContent = p;
      label.className = "block text-sm font-medium text-gray-700";
      
      const input = document.createElement("input");
      input.name = p;
      input.placeholder = p;
      input.className = "w-full border px-3 py-2 rounded-md text-sm";
      
      div.appendChild(label);
      div.appendChild(input);
      container.appendChild(div);
    });
  }

  function addRule() {
    const form = document.getElementById("add_rule_form");
    const data = Object.fromEntries(new FormData(form).entries());
    
    // Basic validation
    if (!data.rule_id || !data.description) {
      return alert("Please fill in all required fields");
    }
    
    if (data.value && !isNaN(data.value)) data.value = parseFloat(data.value);
    data.extra = {};
    
    fetch('/add_rule', {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify(data)
    }).then(res => {
      if (res.ok) {
        alert("✅ Rule added successfully");
        fetchRules();
        form.reset();
      }
      else res.json().then(e => alert("Error: " + (e.detail || "Failed to add rule")));
    });
  }

  function deleteRule() {
    const id = document.getElementById("delete_rule_id").value;
    if (!id) return alert("Please enter a rule ID");
    
    if (!confirm(`Are you sure you want to delete rule ${id}?`)) return;
    
    fetch('/delete_rule/' + id, { method: "DELETE" })
      .then(res => {
        if (res.ok) {
          alert("🗑️ Rule deleted successfully");
          fetchRules();
          document.getElementById("delete_rule_id").value = "";
        }
        else res.json().then(e => alert("Error: " + (e.detail || "Failed to delete rule")));
      });
  }

  // Initialize
  document.addEventListener('DOMContentLoaded', function() {
    updateFunctionFields();
    fetchRules();
  });
</script>
<script>
 const functionColumnsMap = {
  get_customers: ["customer_id", "name", "dob", "city"],
  get_accounts: ["account_id", "account_type", "status", "opening_date"],
  get_transactions: ["transaction_id", "account_id", "amount", "transaction_time"],
  get_cust_acc: [
    "customer_id", "name", "dob", "city", // from customers
    "account_id", "account_type", "status", "opening_date" // from accounts
  ],
  get_cust_tran: [
    "customer_id", "name", "dob", "city", // from customers
    "transaction_id", "amount", "transaction_time" // from transactions
  ],
  get_acc_tran: [
    "account_id", "account_type", "status", "opening_date", // from accounts
    "transaction_id", "amount", "transaction_time" // from transactions
  ],
  get_cust_acc_tran: [
    "customer_id", "name", "dob", "city", // from customers
    "account_id", "account_type", "status", "opening_date", // from accounts
    "transaction_id", "amount", "transaction_time" // from transactions
  ]
};


function populateColumns() {
  const functionSelect = document.getElementById("function_dropdown");
  const columnDropdown = document.getElementById("column_dropdown");
  const selectedFunction = functionSelect.value;

  // Clear current column dropdown
  columnDropdown.innerHTML = '<option value="">Select column</option>';

  // Populate based on selected function
  if (functionColumnsMap[selectedFunction]) {
    functionColumnsMap[selectedFunction].forEach(col => {
      const option = document.createElement("option");
      option.value = col;
      option.textContent = col;
      columnDropdown.appendChild(option);
    });
  }
}

</script>

</body>
</html>
""")

Processing c:\b\abs_5c8w79vk0u\croot\anyio_1745334672105\work (from -r requirements.txt (line 3))


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\b\\abs_5c8w79vk0u\\croot\\anyio_1745334672105\\work'

